In [ ]:
# https://github.com/MicrosoftLearning/mslearn-openai/blob/main/Instructions/Exercises/02-natural-language-azure-openai.md

import os
from dotenv import load_dotenv

# Add Azure OpenAI package
from openai import AzureOpenAI


def main():
    try:
        # Get configuration settings
        load_dotenv()
        azure_oai_key = os.getenv("AZURE_OPENAI_KEY")
        azure_oai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
        azure_oai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
        azure_oai_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")

        # Initialize the Azure OpenAI client
        client = AzureOpenAI(
            api_key=azure_oai_key,
            api_version=azure_oai_api_version,
            azure_endpoint=azure_oai_endpoint,
        )

        # Create a system message
        system_message = "I am a hiking enthusiast named Forest who helps people discover hikes in their area. If no area is specified, I will default to near Rainier National Park. I will then provide three suggestions for nearby hikes that vary in length. I will also share an interesting fact about the local nature on the hikes when making a recommendation."

        # Initialize messages array
        messages_array = [{"role": "system", "content": system_message}]

        while True:
            # Get input text
            input_text = input("Enter the prompt (or type 'quit' to exit):")
            if input_text.lower() == "quit":
                break
            if len(input_text) == 0:
                print("Please enter a prompt.")
                continue
            print("\nSending request for summary to Azure OpenAI endpoint...\n")

            # Send request to Azure OpenAI model
            messages_array.append({"role": "user", "content": input_text})
            response = client.chat.completions.create(
                model=azure_oai_deployment,
                temperature=0.7,
                max_tokens=1500,
                messages=messages_array,
            )

            generated_text = response.choices[0].message.content
            # Add generated text to messages array
            messages_array.append({"role": "assistant", "content": generated_text})

            # Print generated text
            print("Summary: " + generated_text + "\n")
    except Exception as ex:
        print(ex)


if __name__ == "__main__":
    main()

In [ ]:
# https://github.com/MicrosoftLearning/mslearn-openai/blob/main/Instructions/Exercises/06-use-own-data.md

import os
from dotenv import load_dotenv

# Add Azure OpenAI package
from openai import AzureOpenAI


def main():
    try:
        # Get configuration settings
        load_dotenv()
        azure_oai_key = os.getenv("AZURE_OPENAI_KEY")
        azure_oai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
        azure_oai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
        azure_oai_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
        azure_search_key = os.getenv("AZURE_SEARCH_KEY")
        azure_search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
        azure_search_index = os.getenv("AZURE_SEARCH_INDEX")

        # Initialize the Azure OpenAI client
        client = AzureOpenAI(
            api_key=azure_oai_key,
            api_version=azure_oai_api_version,
            azure_endpoint=azure_oai_endpoint,
        )

        # Get the prompt
        text = input("\nEnter a question:\n")

        # Send request to Azure OpenAI model
        print("\nSending the following request to Azure OpenAI endpoint...\n")
        print("Request: " + text + "\n")

        # Ref: https://learn.microsoft.com/en-us/azure/ai-services/openai/use-your-data-quickstart
        response = client.chat.completions.create(
            model=azure_oai_deployment,
            temperature=0.5,
            max_tokens=1000,
            messages=[
                {"role": "system", "content": "You are a helpful travel agent."},
                {"role": "user", "content": text},
            ],
            # Configure your data source
            extra_body={
                "data_sources": [
                    {
                        "type": "azure_search",
                        "parameters": {
                            "authentication": {
                                "type": "api_key",
                                "key": azure_search_key,
                            },
                            "endpoint": azure_search_endpoint,
                            "index_name": azure_search_index,
                        },
                    }
                ],
            },
        )

        # Flag to show citations
        show_citations = True

        # Print response
        print("Response: " + response.choices[0].message.content + "\n")

        if show_citations:
            # Print citations
            print("Citations: ")
            citations = response.choices[0].message.context["citations"]
            for c in citations:
                print("Title: " + c["title"] + "\tURL: " + c["url"])
    except Exception as ex:
        print(ex)


if __name__ == "__main__":
    main()

In [ ]:
# https://github.com/Azure/azure-search-vector-samples/tree/main/demo-python/code/basic-vector-workflow
# Part 1
# Prerequisite: Azure AI Search service with basic plan (same region as Azure OpenAI resource), and Semantic ranker enabled with free plan

import os
import json
from dotenv import load_dotenv
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import (
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchField,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,
    SearchIndex,
)
from azure.search.documents import SearchClient

load_dotenv(override=True)
azure_oai_key = os.getenv("AZURE_OPENAI_KEY")
azure_oai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
azure_oai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_oai_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_oai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL")
azure_search_key = (
    AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))
    if len(os.getenv("AZURE_SEARCH_KEY")) > 0
    else DefaultAzureCredential()
)
azure_search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
azure_search_index_vector = os.getenv("AZURE_SEARCH_INDEX_VECTOR")

azure_oai_client = AzureOpenAI(
    api_key=azure_oai_key,
    api_version=azure_oai_api_version,
    azure_endpoint=azure_oai_endpoint,
)

input_path = os.path.join(".", "text-sample.json")
with open(input_path, "r", encoding="utf-8") as file:
    input_data = json.load(file)

# Generate embeddings for title and content fields
titles = [item["title"] for item in input_data]
title_response = azure_oai_client.embeddings.create(
    input=titles,
    model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
)
title_embeddings = [item.embedding for item in title_response.data]
content = [item["content"] for item in input_data]
content_response = azure_oai_client.embeddings.create(
    input=content,
    model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
)
content_embeddings = [item.embedding for item in content_response.data]

# Add vectorized title and content fields
for i, item in enumerate(input_data):
    title = item["title"]
    content = item["content"]
    item["titleVector"] = title_embeddings[i]
    item["contentVector"] = content_embeddings[i]

output_path = os.path.join(".", "text-sample-vectors.json")
with open(output_path, "w") as f:
    json.dump(input_data, f)

# Initialize the Azure AI Search index client
index_client = SearchIndexClient(
    credential=azure_search_key,
    endpoint=azure_search_endpoint,
)

index_fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        sortable=True,
        filterable=True,
        facetable=True,
    ),
    SearchableField(
        name="title",
        type=SearchFieldDataType.String,
    ),
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
    ),
    SearchableField(
        name="category",
        type=SearchFieldDataType.String,
        filterable=True,
    ),
    SearchField(
        name="titleVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_profile_name="myHnswProfile",
    ),
    SearchField(
        name="contentVector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_profile_name="myHnswProfile",
    ),
]

# Apply the vector search with corresponding settings
vector_search = VectorSearch(
    algorithms=[HnswAlgorithmConfiguration(name="myHnsw")],
    profiles=[
        VectorSearchProfile(
            name="myHnswProfile",
            algorithm_configuration_name="myHnsw",
        )
    ],
)

# Apply the semantic search with corresponding settings
semantic_config = SemanticConfiguration(
    name="mySemanticConfig",
    prioritized_fields=SemanticPrioritizedFields(
        title_field=SemanticField(field_name="title"),
        keywords_fields=[SemanticField(field_name="category")],
        content_fields=[SemanticField(field_name="content")],
    ),
)
semantic_search = SemanticSearch(configurations=[semantic_config])

# Create a new search index (contains a vector store) with related fields, and both vector search and semantic search profiles for hybrid search purpose
index = SearchIndex(
    name=azure_search_index_vector,
    fields=index_fields,
    vector_search=vector_search,
    semantic_search=semantic_search,
)
result = index_client.create_or_update_index(index)
print(f"Index: {result.name} created/updated\n")

# Initialize the Azure AI Search client
search_client = SearchClient(
    credential=azure_search_key,
    endpoint=azure_search_endpoint,
    index_name=azure_search_index_vector,
)

input_path = os.path.join(".", "text-sample-vectors.json")
with open(input_path, "r", encoding="utf-8") as file:
    input_documents = json.load(file)

# Upload the prepared document (title and content fields are vectorized, while the other remains original readable text) and insert into the index
result = search_client.upload_documents(input_documents)
print(f"Uploaded {len(input_documents)} records\n")

In [ ]:
# https://github.com/Azure/azure-search-vector-samples/tree/main/demo-python/code/basic-vector-workflow
# Part 2

from azure.search.documents.models import VectorizedQuery
from azure.search.documents.models import VectorFilterMode
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

# Vectorize plain text query
query = "tools for software development"
query_embedded = (
    azure_oai_client.embeddings.create(
        input=query,
        model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
    )
    .data[0]
    .embedding
)

# Perform a single vector search, with a pre-filter
query_vectorized = VectorizedQuery(
    vector=query_embedded,
    k_nearest_neighbors=3,
    fields="contentVector",
)

print("\nPerform a single vector search...\n")
results = search_client.search(
    search_text=None,
    vector_queries=[query_vectorized],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter="category eq 'Developer Tools'",
    select=["title", "content", "category"],
)
for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

# Perform a cross-field vector search, allows to query multiple vector fields at the same time
query_vectorized = VectorizedQuery(
    vector=query_embedded,
    k_nearest_neighbors=3,
    fields="contentVector, titleVector",
)

print("\nPerform a cross-field search...\n")
results = search_client.search(
    search_text=None,
    vector_queries=[query_vectorized],
    select=["title", "content", "category"],
)
for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

# Perform a multi-vector search, allows to query multiple vector fields at the same time by passing in multiple query vectors
query_vectorized_1 = VectorizedQuery(
    vector=query_embedded,
    k_nearest_neighbors=3,
    fields="titleVector",
)
query_vectorized_2 = VectorizedQuery(
    vector=query_embedded,
    k_nearest_neighbors=3,
    fields="contentVector",
)

print("\nPerform a multi-vector search...\n")
results = search_client.search(
    search_text=None,
    vector_queries=[query_vectorized_1, query_vectorized_2],
    select=["title", "content", "category"],
)
for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

# Perform a hybrid search (keyword + vector)
query = "scalable storage solution"
query_embedded = (
    azure_oai_client.embeddings.create(
        input=query,
        model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
    )
    .data[0]
    .embedding
)

query_vectorized = VectorizedQuery(
    vector=query_embedded,
    k_nearest_neighbors=3,
    fields="contentVector",
)

print("\nPerform a hybrid search (keyword + vector)...\n")
results = search_client.search(
    # Consider pre-processing here like rewriting the query prompt into a summary word, e.g., storage, which might be optimized for keyword or semantic sections
    search_text=query,
    vector_queries=[query_vectorized],
    select=["title", "content", "category"],
    top=3,
)
for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

# Perform a semantic hybrid search (keyword + vector + semantic reranking)
query = "data analytics platform"
query_embedded = (
    azure_oai_client.embeddings.create(
        input=query,
        model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
    )
    .data[0]
    .embedding
)

query_vectorized = VectorizedQuery(
    vector=query_embedded,
    k_nearest_neighbors=20,
    fields="contentVector",
    exhaustive=True,
)

print("\nPerform a semantic hybrid search (keyword + vector + semantic reranking)...\n")
results = search_client.search(
    # Consider pre-processing here like rewriting the query prompt into a summary word, e.g., analytics, which might be optimized for keyword or semantic sections
    search_text=query,
    vector_queries=[query_vectorized],
    select=["title", "content", "category"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name="mySemanticConfig",
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3,
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"Title: {result['title']}")
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}")

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")